### Coursera : Description: Final Project for Course 3 - OMDB and TasteDive Mashup

In [ ]:
import requests_with_caching

def get_movies_from_tastedive(name_artist):
    base_url = "https://tastedive.com/api/similar"
    params_diction = {} # Set up an empty dictionary for query parameters
    params_diction["q"] = name_artist
    params_diction["type"] = "movies"
    params_diction["limit"] = "5" 
    # , permanent_cache_file="this_page_cache.txt"
    res = requests_with_caching.get(base_url, params=params_diction)
    #print(res.url)
    res_dict = res.json()
    return res_dict

def extract_movie_titles(d):
    titles = [d["Name"] for d in d["Similar"]["Results"]]
    #print(titles)
    return titles

def get_related_titles(titles):
    related_titles = []
    f_r_titles = []
    for t in titles:
        new_titles = extract_movie_titles(get_movies_from_tastedive(t))
        for nt in new_titles:
            if nt not in related_titles:
                related_titles.append(nt)
    return related_titles


def get_movie_data(title):
    base_url = "http://www.omdbapi.com/"
    pd = {}
    pd["t"] = title
    pd["r"] = "json"
    
    res = requests_with_caching.get(base_url, params=pd)
    obj = res.json()
    return obj

def get_movie_rating(omdbd):
    ratings_d = omdbd['Ratings']
    rating_val = 0 
    for r in ratings_d:
        if r["Source"] == "Rotten Tomatoes":
            rating_val = int(r['Value'].replace('%',''))
     
    return rating_val

def get_rating_by_title(title):
    return get_movie_rating(get_movie_data(title))

def get_sorted_recommendations(mv_titles_list):
    rt_with_rating = {}
    related_titles = get_related_titles(mv_titles_list)

    for rt in related_titles:
        rt_with_rating[rt] = get_rating_by_title(rt)
    print(rt_with_rating.items())
    sorted_ritems = sorted(rt_with_rating.items(), key=lambda kv:(kv[1],kv[0]), reverse=True)
    result_list = [i[0] for i in sorted_ritems]    
    print(result_list)
    return result_list